### README
This code takes three premade excel file, one with the averages of all players during a season, one with the team averages for the same season,
and one with a specific players statistics for each of their individual games for that season. It then combines information pertaining to that 
specific player into one large excel file. The excel file is being created to train an ML data set. The inputs will be the team/player averages, 
and the output will be the outcomes of the game

In [ ]:
import pandas as pd
import os
from bs4 import BeautifulSoup
import requests
import numpy as np

In [ ]:
def scrape_player_gamelog(player_id, url, year):
    
    #Go to website and open basketball reference page
    response = requests.get(url)

    # Check for response status
    if response.status_code != 200:
        print(f"Failed to retrieve data for {player_id}. Status code: {response.status_code}")
        return None  # Exit function early
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find('table', {'id': 'pgl_basic'})
    headers = [th.text for th in table.find('thead').find_all('th')][1:]
    rows = table.find('tbody').find_all('tr')
    data = []
    # Pull the data from the table 
    for row in rows:
        if row.find('th', {'scope': 'row'}) is not None:
            game_date = row.find('th').text
            cells = row.find_all('td')
            cells_data = [cell.text for cell in cells]
            data.append([player_id, game_date] + cells_data)
    headers = ['Player_ID', 'Date'] + headers
    df = pd.DataFrame(data, columns=headers)
    # Remove rows where the player did not play (G column is empty)
    df = df[df['G'] != '']
    # Modify the 7th column based on its content
    df.iloc[:, 6] = df.iloc[:, 6].apply(lambda x: 'away' if '@' in x else 'home')
    # Rename columns
    df.rename(columns={df.columns[8]: 'Court'}, inplace=True)
    # Remove the second column
    df.drop(df.columns[1], axis=1, inplace=True)
    # Define the directory where you want to save the file
    save_directory = r'C:\Users\kings\Desktop\CS Projects\Game Logs'
    # Ensure the directory exists
    os.makedirs(save_directory, exist_ok=True)
    # Create the full file path
    file_name = os.path.join(save_directory, f'{player_id}_gamelog_{year}.xlsx')
    # Save to Excel with player ID in the file name
    with pd.ExcelWriter(file_name) as writer:
        df.to_excel(writer, index=False, sheet_name='Game Log')

In [ ]:


def all_files_combine(player_id, PLAYER_GAMELOG, PLAYER_AVERAGES, TEAM_AVERAGES, year):
    player_stats_df = pd.read_excel(PLAYER_GAMELOG)
    team_stats_df = pd.read_excel(TEAM_AVERAGES)
    player_averages_df = pd.read_excel(PLAYER_AVERAGES)
    print('Player Averages Columns:', player_averages_df.columns)
    header = ['Player Name','G','Age','Tm','Court','Opp','Out','GS','MP','FG','FGA','FG%','3P','3PA','3P%','FT','FTA','FT%','ORB','DRB','TRB','AST','STL','BLK','TOV','PF','PTS','GmSc','+/-','avg G','avg GS','avg MP','avg FG','avg FGA','avg FG%','avg 3P','avg 3PA','avg 3P%','avg 2P','avg 2PA','avg 2P%','avg FG%','avg FT','avg FTA','avg FT%','avg ORB','avg DRB','avg TRB','avg AST','avg STL','avg BLK','avg TOV','avg PF','avg PTS','E team PTS','E team FGM','E team FGA','E team FG%','E team 3PM','E team 3PA','E team 3P%','E team FTM','E team FTA','E team FT%','E team OR','E team DR','E team REB','E team AST','E team STL','E team BLK','E team TO','E team PF','same team PTS','same team FGM','same team FGA','same team FG%','same team 3PM','same team 3PA','same team 3P%','same team FTM','same team FTA','same team FT%','same team OR','same team DR','same team REB','same team AST','same team STL','same team BLK','same team TO','same team PF']

    # Ensure 'Opp' column is properly set for merging
    player_stats_df['Opp'] = player_stats_df['Opp'].str.strip()

    # Ensure the player_stats_df has enough columns to accommodate new data
    player_averages_columns = player_averages_df.columns[1:]  # Exclude the 'Player' column
    team_averages_columns = team_stats_df.columns[1:]  # Exclude the 'Team Name' column

    # Check if player_stats_df has enough columns to add new data
    required_columns = 52 + len(player_averages_columns) + len(team_averages_columns)
    if player_stats_df.shape[1] < required_columns:
        new_columns = [f"col_{i}" for i in range(player_stats_df.shape[1], required_columns)]
        player_stats_df = player_stats_df.reindex(columns=[*player_stats_df.columns, *new_columns])

    #find the player averages 
    player_averages = player_averages_df[player_averages_df['Player'] == player_id]
    player_row = player_averages.iloc[0, 1:].values  # Get averages starting from the 2nd column
    for i, row in player_stats_df.iterrows(): #loop through and add the averages to the player_df dataframe
        player_stats_df.iloc[i, 29:29 + len(player_row)] = player_row  # Place the averages starting at the 27th column

    # Iterate over the rows of the player stats DataFrame
    for i, row in player_stats_df.iterrows():
        opponent_team = row['Opp']  # Assuming 'Opp' is the column for the opposing team
        team_stats = team_stats_df[team_stats_df['Team Name'] == opponent_team]  # Assuming 'Team Name' is the column for the team name
        
        if not team_stats.empty:
            team_averages = team_stats.iloc[0, 2:].values  # Get the averages starting from the 2nd column
            player_stats_df.iloc[i, 54:54 + len(team_averages)] = team_averages  # Place the averages starting at the 52nd column
    
    for i, row in player_stats_df.iterrows():
        same_team = row['Tm']
        same_team_stats = team_stats_df[team_stats_df['Team Name'] == same_team] 
        if not same_team_stats.empty:
            team_averages = same_team_stats.iloc[0, 2:].values  # Get the averages starting from the 2nd column
            player_stats_df.iloc[i, 72:72 + len(team_averages)] = team_averages  # Place the averages starting at the 52nd column

        # Remove the last 6 columns
    player_stats_df = player_stats_df.iloc[:, :-6]

    # Set a new header list that matches the actual number of columns in the DataFrame
    current_columns = player_stats_df.columns
    new_header = header[:len(current_columns)]
    player_id = player_id.replace(' ', '_')

    # Save the merged DataFrame to a new Excel file with the first row as the header
   
   # output_path = f"C:\\Users\\kings\\Desktop\\CS Projects\\Automated Final ML Datasets\\ML_READY_{player_id}_{year}.xlsx"
    player_stats_df.columns = new_header  # Set the new header
    player_stats_df.to_excel(output_path, index=False)
    print(f"Merged data has been saved to '{output_path}'")
    #print(header.index('avg G'))
    #print(header.index('team PTS'))


    
def extract_link(name):
    first_and_last = name.split()
    first_name = first_and_last[0]
    last_name = first_and_last[1]
    if len(last_name) > 5:
        last_chars = last_name[0:5].lower()
    else:
        last_chars = last_name.lower()
    first_chars = first_name[0:2].lower()
    full_link = last_chars + first_chars + '01'
    return(full_link)

def create_name_list(player_averages):
    excel_all = pd.read_excel(player_averages)
    excel_names = excel_all['Player'].tolist()
    return(excel_names)

def main():
    # Choose the year the datasets should be from
    year = 2024
    weird_names = []
    error_names = []
    player_averages_file = f"C:\\Users\\kings\\Desktop\\CS Projects\\Team and Player Averages\\{year}_Player_Averages.xlsx"
    team_statistics_file = f"C:\\Users\\kings\\Desktop\\Team and Player Averages\\{year}_Team_Averages.xlsx"
    player_names = create_name_list(player_averages_file)

    for player_id in player_names:
        try:
            if len(player_id.split()) >= 3:
                weird_names.append(player_id)
                continue
            else:
                game_log = f"C:\\Users\\kings\\Desktop\\CS Projects\\Game Logs\\{player_id}_gamelog_{year}.xlsx"
                link_id = extract_link(player_id)
                print(link_id)
                url = f'https://www.basketball-reference.com/players/{link_id[0]}/{link_id}/gamelog/{year}'
                print(url)
                scrape_player_gamelog(player_id, url, year)
                all_files_combine(player_id, game_log, player_averages_file, team_statistics_file, year)
        except Exception as e:
            # Capture any error that occurs and move to the next player
            error_names.append((player_id, str(e)))
            print(f"An error occurred with {player_id}: {e}")
    
    print("These names were not automatically created due to their weirdness:")
    print(weird_names)

    if error_names:
        print("The following players encountered errors during processing:")
        for name, error in error_names:
            print(f"{name}: {error}")

main()



achiupr01
https://www.basketball-reference.com/players/a/achiupr01/gamelog/2024
Failed to retrieve data for Precious Achiuwa. Status code: 429
Player Averages Columns: Index(['Player', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P',
       '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST',
       'STL', 'BLK', 'TOV', 'PF', 'PTS'],
      dtype='object')
Merged data has been saved to 'C:\Users\kings\Desktop\CS Projects\Sports Betting Project\Automated Final ML Datasets\ML_READY_Precious_Achiuwa_2024.xlsx'
adebaba01
https://www.basketball-reference.com/players/a/adebaba01/gamelog/2024
Failed to retrieve data for Bam Adebayo. Status code: 429
Player Averages Columns: Index(['Player', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P',
       '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST',
       'STL', 'BLK', 'TOV', 'PF', 'PTS'],
      dtype='object')
Merged data has been saved to 'C:\Users\kings\Desktop\CS Projects\Sports 

KeyboardInterrupt: 